In [21]:
import requests
import re
from bs4 import BeautifulSoup
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Requesting the table of content page

First we need to download the page containing all links for songs for an artist

In [22]:
urls = ['https://www.lyrics.com/artist/James-Vincent-McMorrow/1093507','https://www.lyrics.com/artist/Mumford-%26-Sons/1570809']

In [23]:
artist_name = ""

def download_artist_page(url):

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    r=requests.get(url, headers=headers)
    text = r.text
    soup = BeautifulSoup(text, 'html')

    artist = soup.h1.text.replace(" ", "_")
    global artist_name
    artist_name = str(artist)
    #artist_name = artist_name.replace('%26', '&')
    #print(artist_name)

    with open (str(artist_name) + '_lyrics_page', 'w') as f:
        f.write(text)
    
    """with open ('james_vincent_mcorrow_lyrics_page', 'r') as f:
        my_string = f.read()
    print(my_string)"""
    
    #print(f)
    return soup
    
download_artist_page('https://www.lyrics.com/artist/Mumford-%26-Sons/1570809')

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]--><!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]--><!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en"> <!--<![endif]--><head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Mumford &amp; Sons Lyrics, Songs and Albums | Lyrics.com</title>
<meta content="Mumford &amp; Sons Lyrics - All the great songs and their lyrics from Mumford &amp; Sons on Lyrics.com" name="description"/>
<meta content="Mumford &amp; Sons lyrics, Mumford &amp; Sons song lyrics, Mumford &amp; Sons lyric" name="keywords"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<base href="https://www.lyrics.com/"/>
<script>
s4Prefix = 'https://static.stands4.com';
version = '1.3.87';
</script>
<!-- Bootstrap compiled and minified CSS -->
<link href="ht

In [24]:
print(artist_name)

Mumford_&_Sons


In [25]:
"""def get_artist_name(url):
    soup = download_artist_page(url)
    artist_name = soup.h1.text.replace(" ", "_")
    return artist_name
    
get_artist_name(url)"""

'def get_artist_name(url):\n    soup = download_artist_page(url)\n    artist_name = soup.h1.text.replace(" ", "_")\n    return artist_name\n    \nget_artist_name(url)'

Then we explore the page with beautiful soup to find all links

In [26]:
def get_links(soup):
    page_links = soup.find_all('a')
    song_links = []
    song_names = []
    for link in page_links: 
        if '/lyric/' in link['href']:
            split_name = link.text.split("[")[0].strip()
            if split_name not in song_names:
                song_links.append(link['href'])
                song_names.append(split_name)
            
    print("Song links found")
    print("___________")
    print("")
    return song_links

#get_links(download_artist_page(url))

We then prepare a function to extract the lyrics out of the lyrics page and clean up any style or tag

In [27]:
def get_lyrics(url):
    soup_lyrics = BeautifulSoup(requests.get(url).text, 'html')
    lyrics = soup_lyrics.find_all(id='lyric-body-text')

    for tag in lyrics:
        for minitag in tag:
            if minitag in tag.find_all('a'):
                del minitag['href']
                del minitag['style']
                minitag.extract()
    return lyrics[0].get_text().replace("\n", " ")

We can then use that function inside a container function that can iterate through all the links returned by our get_links() function and use get_lyrics to write the lyrics into a file

In [37]:
def write_lyrics(song_links):
    print('Starting lyrics downloads')
    for counter, link in enumerate(song_links[:2]):
        url = 'https://www.lyrics.com' + link
        response = requests.get(url)
        #print(response)
        artist = link.split('/')[3].replace("+", "_")
        song_title = link.split('/')[4].replace("+", " ")
        song_title = song_title.replace("%27", "'")
        song_title = song_title.replace('%26', "&")
        song_title = song_title.replace('%2C', ',')
        song_title = song_title.replace('%C3%A9', 'é')
        song_title = song_title.replace('%5B', '[')
        song_title = song_title.replace('%5D', ']')
        song_title = song_title.replace('%21', '!')
        song_title = song_title.replace('%28', '(')
        song_title = song_title.replace('%29', ')')
        
        
        with open(f'/Project_4/lyrics/{artist_name}.txt', 'w') as f:
            f.write("\n" + get_lyrics(url))
        print(counter, song_title, response.status_code)

        

#links_list = get_links(soup)
#write_lyrics(links_list)

In [33]:
labels = []
corpus = []

def clean_up_lyrics(artist):
    labels_try = []
    with open ('lyrics/' + artist_name +'.txt', 'r') as f:
        my_lyrics = f.read()
    my_lyrics = my_lyrics.split('\n')
    my_lyrics = [s.lower() for s in my_lyrics]

    clean_lyrics = []

    tokenizer = TreebankWordTokenizer()
    lemmatizer = WordNetLemmatizer()

    for doc in my_lyrics:
        tokens = tokenizer.tokenize(text=doc)
        clean_doc = " ".join(lemmatizer.lemmatize(token) for token in tokens)
        clean_lyrics.append(clean_doc)
    labels_one_artist = [artist] * len(clean_lyrics)
    global labels
    labels.extend(labels_one_artist)
    global corpus
    corpus.extend(clean_lyrics)    
#clean_up_lyrics('James_Vincent_McMorrow')

In [38]:
def run_all(url):
    
    all_lyrics = []
    all_labels = []
    soup = download_artist_page(url)
    links_list = get_links(soup)
    write_lyrics(links_list)
    clean_lyrics = clean_up_lyrics(artist_name)
    
for url in urls:
    run_all(url)

Song links found
___________

Starting lyrics downloads


FileNotFoundError: [Errno 2] No such file or directory: '/Project_4/lyrics/James_Vincent_McMorrow.txt'

In [36]:
pwd

'/Users/em/Documents/Spiced/repos/random-rose-student-code/week_4'

In [ ]:
def predict_text(text):
    
    vectorizer = TfidfVectorizer(stop_words=stopwords)
    X = vectorizer.fit_transform(corpus)
    lr = LogisticRegression()
    lr.fit(X, labels)
    lr.score(X, labels)

    steps = [('tf-idf', TfidfVectorizer(stop_words=stopwords)),
              ('LR', LogisticRegression())
            ]

    pipeline = Pipeline(steps)
    pipeline.fit(corpus, labels)
    pipeline.predict([text])
    pipeline.predict_proba([text])
    
    return pipeline.predict([text])[-1]

In [ ]:
predict_text('from the woods')